In [1]:
import pandas as pd
import numpy as np
#一 数据收集 start 
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train.head()

,user_id,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1374,58,No,Travel_Rarely,605,Sales,21,3,Life Sciences,1,...,3,80,1,29,2,2,1,0,0,0
1,1092,45,No,Travel_Rarely,950,Research & Development,28,3,Technical Degree,1,...,4,80,1,8,3,3,5,4,0,3
2,768,40,No,Travel_Rarely,300,Sales,26,3,Marketing,1,...,2,80,1,8,3,2,7,7,7,5
3,569,36,No,Non-Travel,1434,Sales,8,4,Life Sciences,1,...,2,80,0,10,1,3,10,7,0,9
4,911,25,Yes,Travel_Frequently,599,Sales,24,1,Life Sciences,1,...,4,80,0,1,4,3,1,0,1,0


In [2]:
#二 数据预处理 start 
print(train['Attrition'].value_counts())
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

No     988
Yes    188
Name: Attrition, dtype: int64


In [3]:
#train.isna().sum()#查看是否有空值 Step1，对数据进行探索 
#Step2，去掉无用特征，处理分类特征
train= train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test=test.drop(['EmployeeNumber', 'StandardHours'], axis=1)


In [4]:
#分类特征进行特征值编码：
from sklearn import preprocessing 

attr=['Age','BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for fea in attr:
    lbe=preprocessing.LabelEncoder()
    train[fea]=lbe.fit_transform(train[fea])
    test[fea]=lbe.transform(test[fea])
    lbe_list.append(lbe)
print(lbe_list)

[LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()]


In [5]:
#Step3，模型参数配置x
param = {'boosting_type':'gbdt',
                         'objective' : 'binary', #
                         #'metric' : 'binary_logloss',
                         'metric' : 'auc',
#                          'metric' : 'self_metric',
                         'learning_rate' : 0.01,
                         'max_depth' : 15,
                         'feature_fraction':0.8,
                         'bagging_fraction': 0.9,
                         'bagging_freq': 8,
                         'lambda_l1': 0.6,
                         'lambda_l2': 0,
#                          'scale_pos_weight':k,
#                         'is_unbalance':True
        }

In [6]:
#Step4，模型训练，得出预测结果
from sklearn.model_selection import  train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
X_train,X_valid,Y_train,Y_valid=train_test_split(train.drop('Attrition',axis=1),train['Attrition'],test_size=0.2,random_state=42)

/Users/l.longfeng/miniconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


,user_id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
43,434,13,2,1274,1,9,1,1,1,3,...,4,4,1,13,6,4,13,8,0,8
490,1455,22,2,1322,1,2,4,1,1,3,...,3,4,0,8,2,3,2,2,2,2
765,1185,25,2,1291,1,15,2,1,1,3,...,4,1,1,14,3,3,14,10,6,11
575,1115,16,2,479,1,7,4,3,1,1,...,3,3,0,1,4,1,1,0,0,0
135,269,18,2,1403,1,6,3,1,1,4,...,3,3,1,16,4,3,15,13,10,11


In [10]:
train_data=lgb.Dataset(X_train,label=Y_train)
valid_data=lgb.Dataset(X_valid,label=Y_valid)
model=lgb.train(param,train_data,1000,[train_data,valid_data],early_stopping_rounds=200,categorical_feature=attr,verbose_eval=25)
predict=model.predict(test)
#predict_y=model.predict(X_valid)
#print('LR准确率: %0.4lf' % accuracy_score(predic_y, Y_valid))
#print(predict)
test['Attrition']=predict
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test['Attrition']=predict

/Users/l.longfeng/miniconda3/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Age', 'BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds
[25]	training's auc: 0.933685	valid_1's auc: 0.75412
[50]	training's auc: 0.947107	valid_1's auc: 0.764815
[75]	training's auc: 0.961253	valid_1's auc: 0.777365
[100]	training's auc: 0.972022	valid_1's auc: 0.787733
[125]	training's auc: 0.980276	valid_1's auc: 0.794281
[150]	training's auc: 0.984363	valid_1's auc: 0.801484
[175]	training's auc: 0.989159	valid_1's auc: 0.799847
[200]	training's auc: 0.992527	valid_1's auc: 0.802248
[225]	training's auc: 0.994404	valid_1's auc: 0.804213
[250]	training's auc: 0.99559	valid_1's auc: 0.802903
[275]	training's auc: 0.997072	valid_1's auc: 0.805522
[300]	training's auc: 0.997826	valid_1's auc: 0.807814
[325]	training's auc: 0.998491	valid_1's auc: 0.810215
[350]	training's auc: 0.999048	valid_1's auc: 0.811525
[375]	training's auc: 0.999353	valid_1's auc: 0.814362
[400]	training's auc: 0.999641	valid_1's auc: 0.816108
[425]	training's auc: 0.999829	valid_1's auc: 0.817854
[450]	tr

In [12]:
test[['user_id','Attrition']].to_csv('./Submission.csv')

